In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [20]:
#numpy, pandas, seaborn
import numpy as np
import pandas as pd
import seaborn as sns
import nltk

#nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#matplot
import matplotlib.pyplot as plt

#Tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora

In [17]:
train = pd.read_csv(
    '/kaggle/input/nlp-getting-started/train.csv', 
    usecols=['text', 'target'], 
    dtype={'text': str, 'target': np.int64}
)
test = pd.read_csv(
    '/kaggle/input/nlp-getting-started/test.csv',
    usecols=['text', 'id'], 
    dtype={'text': str, 'target': np.int64}
)

In [23]:
lemmatizer = WordNetLemmatizer()
def txt_preprocess(text):
    text = re.sub(r"\n","",text)
    text = text.lower()
    text = re.sub(r"\d","",text)        #Remove digits
    text = re.sub(r'@\w+','',text) # Remove mentions
    text = re.sub(r'[^\w\s]','',text) #Remove punctuation
    text = re.sub(r'http\S+|www.\S+', '', text) #Remove http
    text = re.sub(r'<.*?>', '', text) # Remove html tags
    
    text = word_tokenize(text)
    text = [word for word in text if word not in stopwords.words('english')]
    
    text = [lemmatizer.lemmatize(word) for word in text]
    return ' '.join(text)

In [24]:
train['text'] = train['text'].apply(txt_preprocess)

In [25]:
test['text'] = test['text'].apply(txt_preprocess)

In [26]:
x_train = train['text']
y_train = train['target'].values
x_test = test['text']

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((6090,), (1523,), (6090,), (1523,))

In [29]:
tfv = TfidfVectorizer(max_features = 2000)
tfv_x_train = tfv.fit_transform(x_train).toarray()
tfv_x_test = tfv.transform(x_test).toarray()
tfv_x_val = tfv.transform(x_val).toarray()
tfv_x_test.shape, tfv_x_train.shape, tfv_x_val.shape

((3263, 2000), (6090, 2000), (1523, 2000))